# osu! | Top mods used in plays

Based on top 100 plays by PP. Generates a graph with all data gathered and sorted.

## Set all variables and install required modules

Type all data when prompted to. You can get an API key from https://osu.ppy.sh/p/api by registering an application (name & URL of the app can be random).

In [ ]:
!pip install plotly
!pip install requests

In [1]:
from IPython.display import clear_output

user_id = input("Enter a username/ID: ")
key = input("Enter your osu! API key: ")
clear_output()
print("Done!")

## Get data from osu! servers

In [ ]:
import requests

url = "https://osu.ppy.sh/api/get_user_best"
limit = 100
params = {"k": key, "u": user_id, "limit": limit}

with requests.get(url, params=params) as r:
    scores = r.json()

## Parse data for plot

In [ ]:
from enum import Enum

class Mod(Enum):
    NoMod          = 0
    NoFail         = 1
    Easy           = 2
    NoVideo        = 4
    Hidden         = 8
    HardRock       = 16
    SuddenDeath    = 32
    DoubleTime     = 64
    Relax          = 128
    HalfTime       = 256
    Nightcore      = 512
    Flashlight     = 1024
    Autoplay       = 2048
    SpunOut        = 4096
    Autopilot      = 8192
    Perfect        = 16384
    Key4           = 32768
    Key5           = 65536
    Key6           = 131072
    Key7           = 262144
    Key8           = 524288
    keyMod         = 1015808
    FadeIn         = 1048576
    Random         = 2097152
    LastMod        = 4194304
    TargetPractice = 8388608
    Key9           = 16777216
    Coop           = 33554432
    Key1           = 67108864
    Key3           = 134217728
    Key2           = 268435456

In [ ]:
"""
Generator yielding value of each bit in an integer 
if it's set + value of LSB no matter what.
"""
def parse_mods(value):
    def bits(n):
        if n == 0:
            yield 0
        while n:
            b = n & (~n+1)
            yield b
            n ^= b
    value = int(value)
    bit_values_gen = bits(value)
    return [Mod(i) for i in bit_values_gen]

result = {}
scores = [parse_mods(i["enabled_mods"]) for i in scores]
for score in scores:
    for mod in score:
        try:
            result[mod.name] += 1
        except KeyError:
            result[mod.name] = 0
            result[mod.name] += 1

## Generate a plot and show it to a user

In [ ]:
import plotly.io as pio

fig = {"data": [{"type": "bar", "y": [i for i in result.values()], "x": [i for i in result.keys()]}], "layout": {"title": {"text": f"Most played mods by {user_id}"}}}
pio.show(fig, renderer="colab")